# Regionalized LCA Calculations

Make sure you are in the right kernel! If you are using DdS teaching infrastructure, that is the `regional` kernel.

In [ ]:
from pathlib import Path
from rasterio.plot import show
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import bw2regional as bwr
import csv
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio

In [ ]:
if "LC IMPACT case study" not in bd.projects:
    bi.restore_project_directory('/srv/data/projects/lcimpact-case-study.tar.gz')

In [ ]:
bd.projects.set_current("LC IMPACT case study")

The keys for our functional units:

In [ ]:
for act in bd.Database("Fuel comparison"):
    print(act.key, act)

The methods we have available:

In [ ]:
[x for x in bd.methods if x[0] == 'LC-IMPACT']

Special activities that we might want to consider separately:

In [ ]:
irrigation = [x.id for x in bd.Database("ecoinvent") if 'irrigation' in x['name']]
crops = [x.id for x in bd.Database("ecoinvent") 
         if x['name'] in ('sugarcane production', 'maize grain production')]

We can look over the extension table input data:

In [ ]:
DATA_DIR = Path("data")

In [ ]:
ls data

In [ ]:
show(rasterio.open(DATA_DIR / "sugarcane_landuse_intensity.tif"))

Define functional units, extension tables, and inclusion/exclusion criteria

In [ ]:
activities = [
    ('Fuel comparison', 'sugarcane transport'),
    ('Fuel comparison', 'maize transport'),
    ('Fuel comparison', 'petrol transport'),
]

lcimpact = {
 ("LC-IMPACT", "Climate Change", "Human Health", "Marginal", "Certain", "100 Years"): None,
 ("LC-IMPACT", "Climate Change", "Human Health", "Marginal", "All", "100 Years"): None,
 ("LC-IMPACT", "Climate Change", "Human Health", "Marginal", "Certain", "Infinite"): None,
 ("LC-IMPACT", "Climate Change", "Human Health", "Marginal", "All", "Infinite"): None,
 ("LC-IMPACT", "Climate Change", "Terrestrial Ecosystems", "Marginal", "Certain", "100 Years"): None,
 ("LC-IMPACT", "Climate Change", "Terrestrial Ecosystems",  "Marginal", "All", "100 Years"): None,
 ("LC-IMPACT", "Climate Change", "Terrestrial Ecosystems",  "Marginal", "Certain", "Infinite"): None,
 ("LC-IMPACT", "Climate Change", "Terrestrial Ecosystems",  "Marginal", "All", "Infinite"): None,
 ("LC-IMPACT", "Climate Change", "Aquatic Ecosystems", "Marginal", "All", "100 Years"): None,
 ("LC-IMPACT", "Climate Change", "Aquatic Ecosystems",  "Marginal", "All", "Infinite"): None, 
 ('LC-IMPACT', 'Land Use', 'Occupation', 'Marginal', 'Certain'): {
     'xt others': 'world-topo-ecoregions-weighted-pop-density',
     'activities': crops,
     'sugarcane': 'world-topo-ecoregions-sugarcane_landuse_intensity',
     'maize': 'world-topo-ecoregions-maize_landuse_intensity',
 },
 ('LC-IMPACT', 'Particulate Matter Formation', 'Marginal', 'All'): {
     'xt others': 'world-topo-particulate-matter-weighted-pop-density',
     'activities': [],
     'sugarcane': 'world-topo-particulate-matter-sugarcane_landuse_intensity',
     'maize': 'world-topo-particulate-matter-maize_landuse_intensity',
 },
 ('LC-IMPACT', 'Particulate Matter Formation', 'Marginal', 'Certain'): {
     'xt others': 'world-topo-particulate-matter-weighted-pop-density',
     'activities': [],
     'sugarcane': 'world-topo-particulate-matter-sugarcane_landuse_intensity',
     'maize': 'world-topo-particulate-matter-maize_landuse_intensity',
 },
 ('LC-IMPACT', 'Water Use', 'Ecosystem Quality', 'Surface Water', 'Marginal', 'All'): {
     'xt others': 'world-topo-watersheds-eq-sw-all-weighted-pop-density',
     'activities': irrigation + crops,
     'sugarcane': 'world-topo-watersheds-eq-sw-all-sugarcane_water_intensity',
     'maize': 'world-topo-watersheds-eq-sw-all-maize_water_intensity',
 },
 ('LC-IMPACT', 'Water Use', 'Ecosystem Quality', 'Surface Water', 'Marginal', 'Certain'): {
     'xt others': 'world-topo-watersheds-eq-sw-certain-weighted-pop-density',
     'activities': irrigation + crops,
     'sugarcane': 'world-topo-watersheds-eq-sw-certain-sugarcane_water_intensity',
     'maize': 'world-topo-watersheds-eq-sw-certain-maize_water_intensity',
 },
 ('LC-IMPACT', 'Water Use', 'Human Health', 'Marginal'): {
     'xt others': 'world-topo-watersheds-hh-weighted-pop-density',
     'activities': irrigation + crops,
     'sugarcane': 'world-topo-watersheds-hh-sugarcane_water_intensity',
     'maize': 'world-topo-watersheds-hh-maize_water_intensity',
 },
}

Let's do some calculations

In [ ]:
func_unit = ('Fuel comparison', 'sugarcane transport')
crop = 'sugarcane'
method = ('LC-IMPACT', 'Land Use', 'Occupation', 'Marginal', 'Certain')

In [ ]:
xt_ag = bwr.ExtensionTablesLCA(
    {func_unit: 1},
    method,
    xtable=lcimpact[method][crop],
    limitations={
        'activities': lcimpact[method]['activities'],
    },
)
xt_ag.lci()
xt_ag.lcia()
xt_ag.score

Explore how impact moves down the supply chain

In [ ]:
ba.utils.print_recursive_calculation(
    activity=('Fuel comparison', 'sugarcane transport'),
    lcia_method=method,
    max_level=4,
    cutoff=2.5e-2,
    _lca_obj=xt_ag,
    _total_score=xt_ag.score,
)

In [ ]:
xt_others = bwr.ExtensionTablesLCA(
    {func_unit: 1},
    method,
    xtable=lcimpact[method]['xt others'],
    limitations={
        'activities': lcimpact[method]['activities'],
        'activities mode': 'exclude'
    }
)
xt_others.lci()
xt_others.lcia()
xt_others.score

In [ ]:
xt_ag.geodataframe_xtable_spatial_scale().explore(column='score_rel')

In [ ]:
xt_ag.geodataframe_xtable_spatial_scale().to_file('xt_scale.gpkg')

In [ ]:
xt_ag.geodataframe_xtable_spatial_scale(sum_flows=False).to_file('xt_scale_unsummed.gpkg')

In [ ]:
xt_ag.geodataframe_xtable_spatial_scale(annotate_flows=True).to_file('xt_scale_annotated.gpkg')

In [ ]:
xt_ag.geodataframe_ia_spatial_scale().to_file('ia_scale.gpkg')

In [ ]:
xt_ag.geodataframe_inv_spatial_scale().to_file('inv_scale.gpkg')

In [ ]:
xt_ag.characterized_inventory += xt_others.characterized_inventory

In [ ]:
xt_ag = bwr.ExtensionTablesLCA(
    {func_unit: 1},
    method,
    xtable=lcimpact[method][crop],
    limitations={
        'activities': lcimpact[method]['activities'],
    },
    use_distributions=True
)
xt_ag.lci()
xt_ag.lcia()
xt_ag.score

In [ ]:
for _ in range(10):
    next(xt_ag)
    print(xt_ag.score)